<a href="https://colab.research.google.com/github/DataSavvyYT/AI-engineering-course/blob/main/05_agentic_ai/01_agentic_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai

In [2]:
import os
import json
import getpass
from openai import OpenAI

In [4]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [5]:
# 1. SETUP
# Enter your key when prompted
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
client = OpenAI()

In [6]:
# 2. THE TOOL (The "Hand" of the Agent)
# This is a simple Python function that returns hardcoded data.
# In real life, this would hit a weather API.
def get_current_weather(location):
    print(f"   [System] accessing weather database for {location}...")
    if "london" in location.lower():
        return json.dumps({"temperature": "15°C", "status": "Rainy"})
    elif "paris" in location.lower():
        return json.dumps({"temperature": "22°C", "status": "Sunny"})
    else:
        return json.dumps({"temperature": "unknown", "status": "unknown"})

In [7]:
# 3. THE TOOL DEFINITION (The "Manual" for the LLM)
# This JSON tells the LLM exactly what the tool does and how to use it.
tools_schema = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather for a specific city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                },
                "required": ["location"],
            },
        },
    }
]

In [10]:
# 4. THE AGENT LOOP
def run_simple_agent(user_question):
    print(f"User asks: '{user_question}'")

    # Message History
    messages = [{"role": "user", "content": user_question}]

    # Call 1: Ask the LLM (It sees the question and the tool definitions)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools_schema,
        tool_choice="auto",
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # CHECK: Did the LLM decide it needs to use a tool?
    if tool_calls:
        print("Agent thought: 'I need to use the weather tool for this.'")

        # Identify which function to call
        tool_call_id = tool_calls[0].id
        function_name = tool_calls[0].function.name
        function_args = json.loads(tool_calls[0].function.arguments)

        # Execute the function
        if function_name == "get_current_weather":
            function_response = get_current_weather(
                location=function_args.get("location")
            )

        # Call 2: Feed the tool output BACK to the LLM
        # We append the LLM's original request AND the tool's answer
        messages.append(response_message)
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tool_call_id,
                "name": function_name,
                "content": function_response,
            }
        )

        print("Agent thought: 'Okay, I have the data. Now I will answer the user.'")

        final_response = client.chat.completions.create(
            model="gpt-40-mini",
            messages=messages,
        )

        return final_response.choices[0].message.content
    else:
        return response_message.content

In [11]:
# --- RUN IT ---
answer = run_simple_agent("What is the weather like in London right now?")
print(f"\nFinal Answer: {answer}")

User asks: 'What is the weather like in London right now?'


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}